### 基于灰狼优化的遗传编程
**GP入门系列教程地址：https://github.com/hengzhe-zhang/DEAP-GP-Tutorial**

**前言：本文章旨在帮助研究灰狼算法的同学了解遗传编程，以及如何将灰狼算法应用到遗传编程中。文章不代表课题组学术观点。**

**灰狼优化**和**灰狼**的关系就和**蚂蚁上树**与**蚂蚁**的关系是一样的。灰狼优化里面当然没有灰狼，正如蚂蚁上树里面也不会真的有蚂蚁一样。

所谓灰狼优化，即Seyedali Mirjalili观察到灰狼种群分为alpha, beta, delta和omega狼，**alpha, beta, delta会带领omega狼**，从而设计的一种优化算法。

灰狼算法现在有14000+的引用量，应该说还算是一个比较有影响力的算法。

![灰狼优化](img/greywolfga.jpg)

### 实验问题

本文的实验问题是GP领域最经典的符号回归问题，即根据训练数据，找到真实函数。

在这里，我们的真实函数是$x^3 + x^2$。

In [545]:
import math
import operator
import random

import numpy as np
from deap import base, creator, tools, gp
from deap.tools import selTournament

np.random.seed(0)
random.seed(0)


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    # 计算均方误差（Mean Square Error，MSE）
    mse = ((func(x) - (x ** 3 + x ** 2)) ** 2 for x in range(-10, 10))
    return (math.fsum(mse),)


# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

#### 选择算子
经典的灰狼算法主要是用于优化连续优化问题，对于遗传编程，我们可以基于遗传编程算法的特点，稍加修改。

在这里，我们将Top-3的个体作为alpha, beta, delta，剩下的个体作为omega。

然后，我们随机选择alpha, beta, delta中的一个个体，或者omega中的一个个体，作为新一代的个体。

这里，由于选择alpha, beta, delta的概率是0.5，因此相当于整个种群会被alpha, beta, delta个体引领。这也就是灰狼算法最核心的思想。

In [546]:
from operator import attrgetter


def selGWO(individuals, k, fit_attr="fitness"):
    # 根据适应度对个体进行排序；最优个体排在前面
    sorted_individuals = sorted(individuals, key=attrgetter(fit_attr), reverse=True)

    # 确定Top-3个体（alpha, beta, delta）
    leaders = sorted_individuals[:3]

    # 剩余的个体被视为omega
    omega = sorted_individuals[3:]

    # 选择交叉/变异的个体
    return [random.choice(leaders) if random.random() < 0.5 else random.choice(omega) for _ in range(k)]

In [547]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=0, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", selGWO)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

### 实际结果

现在，可以运行一下，看看实际的结果。

In [548]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=100)
hof = tools.HallOfFame(1)
_ = algorithms.eaSimple(population=population,
                        toolbox=toolbox, cxpb=0.9, mutpb=0.1, ngen=10, stats=mstats, halloffame=hof,
                        verbose=True)

   	      	                              fitness                              	                      size                     
   	      	-------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg        	gen	max        	min	nevals	std        	avg  	gen	max	min	nevals	std    
0  	100   	1.85626e+09	0  	1.63558e+11	20 	100   	1.63417e+10	13.01	0  	109	1  	100   	18.8199
1  	97    	2.78507e+11	1  	2.77639e+13	20 	97    	2.7624e+12 	16.59	1  	84 	1  	97    	14.7764
2  	91    	1.45053e+10	2  	1.44943e+12	0  	91    	1.44215e+11	18.93	2  	66 	1  	91    	8.99139
3  	92    	1.05095e+14	3  	1.0508e+16 	0  	92    	1.04554e+15	22.02	3  	115	1  	92    	13.124 
4  	90    	3.5126e+08 	4  	1.76023e+10	0  	90    	2.43413e+09	21.97	4  	43 	1  	90    	7.74914
5  	88    	3.5427e+08 	5  	1.76023e+10	0  	88    	2.40632e+09	24.41	5  	43 	6  	88    	6.87036
6  	91    	3.62014e+08	6  	1.76022e+10	0  	91    	2.4341e+09 	26.1 	6  	98 	4  	9

In [549]:
print(str(hof[0]))

add(add(neg(x), mul(mul(x, x), sub(x, -1))), neg(mul(add(0, x), neg(1))))


In [550]:
toolbox.register("select", selTournament, tournsize=3)
population = toolbox.population(n=100)
hof = tools.HallOfFame(1)
_ = algorithms.eaSimple(population=population,
                        toolbox=toolbox, cxpb=0.9, mutpb=0.1, ngen=10, stats=mstats, halloffame=hof,
                        verbose=True)

   	      	                              fitness                              	                      size                     
   	      	-------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg        	gen	max       	min   	nevals	std        	avg  	gen	max	min	nevals	std    
0  	100   	5.51171e+12	0  	5.5117e+14	154094	100   	5.48407e+13	12.95	0  	100	1  	100   	20.4931
1  	95    	2.67999e+06	1  	3.36682e+06	154094	95    	426386     	7.25 	1  	59 	1  	95    	12.0361
2  	91    	2.65683e+06	2  	1.11493e+07	154094	91    	1.01201e+06	9.33 	2  	62 	1  	91    	14.128 
3  	88    	2.64718e+06	3  	2.828e+07  	40666 	88    	2.87378e+06	17.97	3  	100	1  	88    	21.0701
4  	91    	2.52386e+10	4  	2.40504e+12	670   	91    	2.39326e+11	33.98	4  	104	1  	91    	25.6355
5  	95    	4.33015e+06	5  	2.3502e+08 	670   	95    	2.3649e+07 	52.02	5  	142	3  	95    	25.333 
6  	86    	1.46688e+12	6  	1.46688e+14	20    	86    	1.45952e+

In [551]:
print(str(hof[0]))

sub(neg(sub(sub(neg(add(1, add(1, x))), mul(add(1, x), mul(x, x))), mul(neg(neg(x)), -1))), sub(sub(neg(sub(add(-1, x), mul(1, x))), -1), neg(neg(0))))


从结果可以看出，灰狼优化和传统的Tournament算子都可以成功地找到真实函数。相比之下，灰狼优化可以在更少的迭代次数内找到真实函数。